In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


**1. Data Processing:** This data set is a bit messy, so the preprocessing portion is largely a tutorial to make sure students have data ready for keras. 

a) Import the following libraries: 

In [0]:
import sys
import os
import json
import pandas
import numpy
import optparse

from keras.callbacks import TensorBoard
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from collections import OrderedDict

Using TensorFlow backend.


b) We will read the code in slightly differently than before: 

In [0]:
dataframe = pandas.read_csv('/content/gdrive/My Drive/Machine Learning/Assignment 7/dev-access.csv', engine='python', quotechar='|', header=None)

c) We then need to convert to a numpy.ndarray type: 

In [0]:
dataset = dataframe.values

d) Check the shape of the data set - it should be (26773, 2). Spend some time looking at the data. 

In [0]:
dataset.shape

(26773, 2)

e) Store all rows and the 0th index as the feature data: 

In [0]:
X = dataset[:,0]

f) Store all rows and index 1 as the target variable: 

In [0]:
Y = dataset[:,1]

g) In the next step, we will clean up the predictors. This includes removing features that are not valuable, such as timestamp and source. 

In [0]:
for index, item in enumerate(X):
# Quick hack to space out json elements
  reqJson = json.loads(item, object_pairs_hook=OrderedDict)
  del reqJson['timestamp']
  del reqJson['headers']
  del reqJson['source']
  del reqJson['route']
  del reqJson['responsePayload']
  X[index] = json.dumps(reqJson, separators=(',', ':'))

h) We next will tokenize our data, which just means vectorizing our text. Given the data we will tokenize every character (thus char_level = True)

In [0]:
tokenizer = Tokenizer(filters='\t\n', char_level=True)
tokenizer.fit_on_texts(X)

# we will need this later
num_words = len(tokenizer.word_index)+1
X = tokenizer.texts_to_sequences(X)

i) Need to pad our data as each observation has a different length

In [0]:
max_log_length = 1024
X_processed = sequence.pad_sequences(X, maxlen=max_log_length)

j) Create your train set to be 75% of the data and your test set to be 25%

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_processed, Y, test_size=0.25)

**2. Model 1 - RNN:** The first model will be a pretty minimal RNN with only an embedding layer, LSTM layer, and Dense layer. The next model we will add a few more layers. 

a) Start by creating an instance of a Sequential model

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Activation

model = Sequential()

b) From there, add an Embedding layer: 

Params:
- input_dim = num_words (the variable we created above)
- output_dim = 32
- input_length = max_log_length (we also created this above) 
- Keep all other variables as the defaults (shown below)

In [0]:
model.add(Embedding(input_dim = num_words, output_dim = 32, input_length=max_log_length))

c) Add an LSTM layer 

Params:
- units = 64
- recurrent_dropout=0.5

In [0]:
model.add(LSTM(units=64,recurrent_dropout=0.5))

d) Finally, we will add a Dense layer:

Params:
- units = 1 (this will be our output)
- activation = relu

In [0]:
model.add(Dense(units=1,activation='relu'))

e) Compile model using the .compile() method:

Params:
- loss = binary_crossentropy
- optimizer = adam
- metrics = accuracy

In [0]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

f) Print the model summary

In [0]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1024, 32)          2016      
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                24832     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 26,913
Trainable params: 26,913
Non-trainable params: 0
_________________________________________________________________
None


g) Use the .fit() method to fit the model on the train data. Use a validation split of 0.25, epochs=3 and batch size = 128.

In [0]:
model_fit = model.fit(X_train,y_train,epochs=3,batch_size=128)

Epoch 1/3
20079/20079 [==============================] - 254s 13ms/step - loss: 0.9321 - acc: 0.5292
Epoch 2/3
20079/20079 [==============================] - 251s 13ms/step - loss: 0.6217 - acc: 0.6098
Epoch 3/3
20079/20079 [==============================] - 252s 13ms/step - loss: 0.5783 - acc: 0.5844


h) Use the .evaluate() method to get the loss value & the accuracy value on the test data. Use a batch size of 128 again.

In [0]:
score = model.evaluate(X_test, y_test, batch_size=128)

6694/6694 [==============================] - 19s 3ms/step


In [0]:
print("Loss Value:",score[0])
print("Accuracy:",score[1])

Loss Value: 0.5585686050139508
Accuracy: 0.5519868539613434


**3) Model 2 - RNN + Dropout Layers + New Activation Function:**

Now we will add a few new layers to our RNN and switch the activation function. You will be creating a new model here, so make sure to call it something different than the model from Part 2.

a) This RNN needs to have the following layers (add in this order):

- Embedding Layer (use same params as before)
- Dropout Layer (https://keras.io/layers/core/#dropout (Links to an external site.)Links to an external site.) - use a value of 0.5 for now
- LSTM Layer (use same params as before)
- Dropout Layer - use a value of 0.5 
- Dense Layer - (switch to a sigmoid activation function)

In [0]:
model2 = Sequential()
model2.add(Embedding(input_dim = num_words, output_dim = 32, input_length=max_log_length))
model2.add(Dropout(0.5))
model2.add(LSTM(units=64,recurrent_dropout=0.5))
model2.add(Dropout(0.5))
model2.add(Dense(units=1,activation='sigmoid'))

b) Compile model using the .compile() method:

Params:
- loss = binary_crossentropy
- optimizer = adam
- metrics = accuracy

In [0]:
model2.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

c) Print the model summary

In [0]:
print(model2.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1024, 32)          2016      
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024, 32)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                24832     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 26,913
Trainable params: 26,913
Non-trainable params: 0
_________________________________________________________________
None


d) Use the .fit() method to fit the model on the train data. Use a validation split of 0.25, epochs=3 and batch size = 128

In [0]:
model2_fit = model2.fit(X_train,y_train,epochs=3,batch_size=128)

Epoch 1/3
20079/20079 [==============================] - 264s 13ms/step - loss: 0.5989 - acc: 0.6523
Epoch 2/3
20079/20079 [==============================] - 262s 13ms/step - loss: 0.3605 - acc: 0.8630
Epoch 3/3
20079/20079 [==============================] - 262s 13ms/step - loss: 0.3681 - acc: 0.8648


e) Use the .evaluate() method to get the loss value & the accuracy value on the test data. Use a batch size of 128 again

In [0]:
score2 = model2.evaluate(X_test, y_test, batch_size=128)
print("Loss Value:",score2[0])
print("Accuracy:",score2[1])

6694/6694 [==============================] - 19s 3ms/step
Loss Value: 0.2067657412166983
Accuracy: 0.9403943829227285


**4) Recurrent Neural Net Model 3: Build Your Own**

You wil now create your RNN based on what you have learned from Model 1 & Model 2:

a) RNN Requirements: 
- Use 5 or more layers
- Add a layer that was not utilized in Model 1 or Model 2 (Note: This could be a new Dense layer or an additional LSTM)

In [0]:
from keras.layers import Flatten
model3 = Sequential()
model3.add(Embedding(input_dim = num_words, output_dim = 32, input_length=max_log_length))
model3.add(Dense(64,activation='relu'))
model3.add(Dropout(0.5))
model3.add(Dense(64,activation='relu'))
model3.add(Dropout(0.5))
model3.add(Flatten())
model3.add(Dense(1, activation='sigmoid'))

b) Compiler Requirements: 
- Try a new optimizer for the compile step
- Keep accuracy as a metric (feel free to add more metrics if desired)

In [0]:
model3.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

c) Print the model summary

In [0]:
print(model3.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 1024, 32)          2016      
_________________________________________________________________
dense_3 (Dense)              (None, 1024, 64)          2112      
_________________________________________________________________
dropout_3 (Dropout)          (None, 1024, 64)          0         
_________________________________________________________________
dense_4 (Dense)              (None, 1024, 64)          4160      
_________________________________________________________________
dropout_4 (Dropout)          (None, 1024, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 65536)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 65537     
Total para

d) Use the .fit() method to fit the model on the train data. Use a validation split of 0.25, epochs=3 and batch size = 128.

In [0]:
model3_fit = model3.fit(X_train,y_train,epochs=3,batch_size=128)

Epoch 1/3
20079/20079 [==============================] - 67s 3ms/step - loss: 0.3651 - acc: 0.8351
Epoch 2/3
20079/20079 [==============================] - 67s 3ms/step - loss: 0.1156 - acc: 0.9577
Epoch 3/3
20079/20079 [==============================] - 67s 3ms/step - loss: 0.0565 - acc: 0.9814


e) Use the .evaluate() method to get the loss value & the accuracy value on the test data. Use a batch size of 128 again.

In [0]:
score3 = model3.evaluate(X_test, y_test, batch_size=128)
print("Loss Value:",score3[0])
print("Accuracy:",score3[1])

6694/6694 [==============================] - 5s 735us/step
Loss Value: 0.03621236315903635
Accuracy: 0.9855094114132058


In [0]:
X_train.max()

62

**Conceptual Questions:** 

**5) Explain the difference between the relu activation function and the sigmoid activation function. **

The sigmoid function is nonlinear and bounded for 0 to 1 whereas the ReLU is 0 for all negative values and linear with a slope of one for positive values.

**6) In regards to question 5, which of these activation functions performed the best (they were used in Model 1 & Model 2) ? Why do you think that is?**

We see that Model 2 which involves sigmoid activation performs much better than Model 1 using ReLU activation. This makes sense that the sigmoid activation would more accurately predict the output because the output variable of our data is a predictor of 0 or 1 but the input value ranges from 0 to 62.

**7) Explain how dropout works (you can look at the keras code) for (a) training, and (b) test data sets.**

Dropout helps prevent overfitting the data by randomly leaving out certain neurons during the training of the model.

**8) Explain why problems such as this are better modeled with RNNs than CNNs.**

Convolutional Neural Networks are used primarily to map image data or problems working with spacial input values. An RNN Model is more relevent for this problem because is very effective for mapping sequentional data to predictions.

**9) Explain what RNN problem is solved using LSTM and briefly describe how.**

LSTMs are capable of modeling long term dependencies in the data. Therefore it can capture information that would have been left out of the original RNN model

In [0]:
!jupyter nbconvert --to html '/content/gdrive/My Drive/Machine Learning/Assignment 7/Assignment 7.ipynb';

[NbConvertApp] Converting notebook /content/gdrive/My Drive/Machine Learning/Assignment 7/Assignment 7.ipynb to html
[NbConvertApp] Writing 326647 bytes to /content/gdrive/My Drive/Machine Learning/Assignment 7/Assignment 7.html
